## Randomly select 600 new faces

In [76]:
import os
import random
import shutil
from pathlib import Path

SOURCE_DIR = Path("/Users/gauravsharan/Capstone/madde/datasets/Real_ndf")
DEST_DIR = Path("/Users/gauravsharan/Capstone/madde/datasets/Real_50")
SAMPLE_SIZE = 50
EXTENSIONS = {'.png', '.jpg', '.jpeg', '.webp'}  # Allowed image types

def get_image_files(folder):
    return [f for f in folder.iterdir() if f.suffix.lower() in EXTENSIONS and f.is_file()]

def sample_and_copy_images():
    for root, dirs, files in os.walk(SOURCE_DIR):
        root_path = Path(root)
        relative_path = root_path.relative_to(SOURCE_DIR)
        dest_path = DEST_DIR / relative_path

        images = get_image_files(root_path)
        if not images:
            continue

        selected = random.sample(images, min(SAMPLE_SIZE, len(images)))
        dest_path.mkdir(parents=True, exist_ok=True)

        for img_path in selected:
            shutil.copy2(img_path, dest_path / img_path.name)

        print(f"✅ Copied {len(selected)} images to {dest_path}")


sample_and_copy_images()


✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Male/SouthAsian
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Male/EastAsian
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Male/White
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Male/Black
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Female/SouthAsian
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Female/EastAsian
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Female/White
✅ Copied 50 images to /Users/gauravsharan/Capstone/madde/datasets/Real_50/Female/Black


In [6]:
import os
import shutil
import random

def move_images(source_dir, dest_dir, move_counts, seed=42):
    random.seed(seed)

    for subdir, num_to_move in move_counts.items():
        source_subdir = os.path.join(source_dir, subdir)
        dest_subdir = os.path.join(dest_dir, subdir)
        os.makedirs(dest_subdir, exist_ok=True)

        images = [f for f in os.listdir(source_subdir) if os.path.isfile(os.path.join(source_subdir, f))]

        if num_to_move > len(images):
            raise ValueError(f"Requested {num_to_move} files from {source_subdir}, but only {len(images)} available.")

        selected_images = random.sample(images, num_to_move)

        print(f"Moving {num_to_move} images from {source_subdir} to {dest_subdir}")

        for img_name in selected_images:
            src_path = os.path.join(source_subdir, img_name)
            dest_path = os.path.join(dest_subdir, img_name)
            shutil.move(src_path, dest_path)

# Example usage
source_folder = '/home/ec2-user/madde/datasets/Real_4_k_split_copy'
destination_folder = '/home/ec2-user/madde/datasets/Real_5_k_split'
move_counts = {
    'Train': 700,
    'Test': 150,
    'Validation': 150
}

move_images(source_folder, destination_folder, move_counts)


Moving 700 images from /home/ec2-user/madde/datasets/Real_4_k_split_copy/Train to /home/ec2-user/madde/datasets/Real_5_k_split/Train
Moving 150 images from /home/ec2-user/madde/datasets/Real_4_k_split_copy/Test to /home/ec2-user/madde/datasets/Real_5_k_split/Test
Moving 150 images from /home/ec2-user/madde/datasets/Real_4_k_split_copy/Validation to /home/ec2-user/madde/datasets/Real_5_k_split/Validation


## Create Real Dataset for Training

In [12]:
base_combination = ["StyleGAN2_split", "Dall-E_split", "firefly_split", "Midjourney_split", "StableDiffusion_split"]
real_combination = ["Real_1-5_k_split", "Real_2-5_k_split", "Real_3-5_k_split", "Real_4-5_k_split", "Real_5-5_k_split"]
experiment_queue = deque(base_combination)
num_cycles  = len(base_combination)

In [15]:

# Iterate through the combinations in cyclic order
loop = 15
for i in range(num_cycles):
    dataset_type = ""
    for id, j in enumerate(experiment_queue):
        dataset_type += j.replace("_split", "", 1)+"+"
        print(dataset_type, real_combination[id])
    print("###############")
    experiment_queue.rotate(-1)

StyleGAN2+ Real_1-5_k_split
StyleGAN2+Dall-E+ Real_2-5_k_split
StyleGAN2+Dall-E+firefly+ Real_3-5_k_split
StyleGAN2+Dall-E+firefly+Midjourney+ Real_4-5_k_split
StyleGAN2+Dall-E+firefly+Midjourney+StableDiffusion+ Real_5-5_k_split
###############
Dall-E+ Real_1-5_k_split
Dall-E+firefly+ Real_2-5_k_split
Dall-E+firefly+Midjourney+ Real_3-5_k_split
Dall-E+firefly+Midjourney+StableDiffusion+ Real_4-5_k_split
Dall-E+firefly+Midjourney+StableDiffusion+StyleGAN2+ Real_5-5_k_split
###############
firefly+ Real_1-5_k_split
firefly+Midjourney+ Real_2-5_k_split
firefly+Midjourney+StableDiffusion+ Real_3-5_k_split
firefly+Midjourney+StableDiffusion+StyleGAN2+ Real_4-5_k_split
firefly+Midjourney+StableDiffusion+StyleGAN2+Dall-E+ Real_5-5_k_split
###############
Midjourney+ Real_1-5_k_split
Midjourney+StableDiffusion+ Real_2-5_k_split
Midjourney+StableDiffusion+StyleGAN2+ Real_3-5_k_split
Midjourney+StableDiffusion+StyleGAN2+Dall-E+ Real_4-5_k_split
Midjourney+StableDiffusion+StyleGAN2+Dall-E+firefl

In [49]:
REAL_DIR = '/Users/gauravsharan/Capstone/madde/datasets/Real'
DRY_RUN = False  # Set to True to preview only

count = 0
skipped = 0

for root, dirs, files in os.walk(REAL_DIR):
    for file in files:
        if file.startswith('._'):
            file_path = os.path.join(root, file)
            try:
                if DRY_RUN:
                    print(f"[DRY-RUN] Would delete: {file_path}")
                else:
                    os.remove(file_path)
                    print(f"🗑️ Deleted: {file_path}")
                    count += 1
            except Exception as e:
                print(f"❌ Failed to delete {file_path}: {e}")
        else:
            skipped += 1

print("\n✅ Finished cleanup.")
print(f"🔹 Deleted: {count} unwanted ._ files")
print(f"🔸 Skipped: {skipped} other files")


✅ Finished cleanup.
🔹 Deleted: 0 unwanted ._ files
🔸 Skipped: 0 other files


In [20]:
import os
import shutil
from pathlib import Path

FOLDER_A = Path("/home/ec2-user/madde/datasets/Real_4_k_split")  # Source
FOLDER_B = Path("/home/ec2-user/madde/datasets/Real_5_k_split")  # Destination

def merge_folders():
    for root, dirs, files in os.walk(FOLDER_A):
        root_path = Path(root)
        relative_path = root_path.relative_to(FOLDER_A)
        dest_path = FOLDER_B / relative_path

        # Make sure the destination subfolder exists
        dest_path.mkdir(parents=True, exist_ok=True)

        for file in files:
            src_file = root_path / file
            dst_file = dest_path / file

            # Overwrite if exists (you can skip or rename if needed)
            shutil.copy2(src_file, dst_file)
            print(f"📁 Copied {src_file} → {dst_file}")

merge_folders()


📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/26371_man_indian.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/26371_man_indian.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/15540_man_asian.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/15540_man_asian.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/15875_woman_indian.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/15875_woman_indian.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/18187_man_black.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/18187_man_black.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/11122_woman_asian.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/11122_woman_asian.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/22255.png → /home/ec2-user/madde/datasets/Real_5_k_split/Test/22255.png
📁 Copied /home/ec2-user/madde/datasets/Real_4_k_split/Test/10402_woman_asian.png → /home/ec2-user/madd

In [77]:
import os
from pathlib import Path

SOURCE_DIR = Path("/Users/gauravsharan/Capstone/madde/datasets/Real_50")  # Source
DEST_DIR = Path("/Users/gauravsharan/Capstone/madde/datasets/Real_ndf")  # Destination

def delete_matching_images():
    deleted = 0
    for root, _, files in os.walk(SOURCE_DIR):
        root_path = Path(root)
        relative_path = root_path.relative_to(SOURCE_DIR)

        for file in files:
            source_file = root_path / file
            dest_file = DEST_DIR / relative_path / file

            if dest_file.exists():
                dest_file.unlink()
                deleted += 1
                print(f"🗑️ Deleted: {dest_file}")

    print(f"\n✅ Finished: {deleted} files deleted.")

if __name__ == "__main__":
    delete_matching_images()


🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/07106_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/19938_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/36500_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/39719_man_black.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/22632.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/37257_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/34175_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/11450_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/24539_man_indian.png
🗑️ Deleted: /Users/gauravsharan/Capstone/madde/datasets/Real_ndf/Male/SouthAsian/63937_man_bla

In [20]:
import os
import shutil

base_dirs = {
    '1k': '/home/ec2-user/madde/datasets/Real_1k_split',
    '4k': '/home/ec2-user/madde/datasets/Real_4k_split',
    '5k': '/home/ec2-user/madde/datasets/Real_5_k_split'
}

subfolders = ['Train', 'Test', 'Validation']
target_counts = {'Train': 2800, 'Test': 600, 'Validation': 600}

for subfolder in subfolders:
    path_1k = os.path.join(base_dirs['1k'], subfolder)
    path_4k = os.path.join(base_dirs['4k'], subfolder)
    path_5k = os.path.join(base_dirs['5k'], subfolder)

    files_1k = set(os.listdir(path_1k))
    files_4k = set(os.listdir(path_4k))
    files_5k = set(os.listdir(path_5k))

    # Find files in 5k that are NOT in 1k or 4k
    candidate_files = files_5k - files_1k - files_4k

    # Calculate how many files we need to reach the target
    current_count = len(files_4k)
    needed_count = target_counts[subfolder] - current_count

    if needed_count <= 0:
        print(f"[{subfolder}] Already has {current_count}, no files needed.")
        continue

    # Select needed files
    files_to_copy = list(candidate_files)[:needed_count]

    # Copy the files over
    for fname in files_to_copy:
        src_path = os.path.join(path_5k, fname)
        dest_path = os.path.join(path_4k, fname)
        shutil.copy2(src_path, dest_path)

    print(f"[{subfolder}] Copied {len(files_to_copy)} files to Real_4k_split.")


[Train] Copied 700 files to Real_4k_split.
[Test] Copied 150 files to Real_4k_split.
[Validation] Copied 150 files to Real_4k_split.


## Sanity Check

In [16]:
import os
from pathlib import Path
from collections import defaultdict

ROOT_DIR = Path("/home/ec2-user/madde/datasets/Real_4k_split")
EXTENSIONS = {'.png'} #to filter

def count_files_by_subfolder():
    counts = defaultdict(int)

    for root, _, files in os.walk(ROOT_DIR):
        root_path = Path(root)
        if EXTENSIONS:
            files = [f for f in files if Path(f).suffix.lower() in EXTENSIONS]
        counts[root_path.relative_to(ROOT_DIR)] += len(files)

    # Sort and print
    for subfolder, count in sorted(counts.items()):
        print(f"{subfolder}: {count} files")

count_files_by_subfolder()


.: 0 files
Test: 450 files
Train: 2100 files
Validation: 450 files


In [ ]:
# Check missing files

import os

FOLDER_A = "/Users/gauravsharan/Capstone/madde/datasets/Stylegan2_400/Real"
FOLDER_B = "/Users/gauravsharan/Capstone/madde/datasets/Stylegan2_400/Synth"

missing = []
matched = []

for file in os.listdir(FOLDER_A):
    if not os.path.isfile(os.path.join(FOLDER_A, file)):
        continue

    name, ext = os.path.splitext(file)
    expected_filename = f"{name}{ext}"
    expected_path = os.path.join(FOLDER_B, expected_filename)

    if os.path.exists(expected_path):
        matched.append(file)
    else:
        missing.append(file)

print(f"\n✅ Comparison complete.")
print(f"✔️  Matched: {len(matched)} files: {matched}")
print(f"❌ Missing: {len(missing)} files: {missing}")

if missing:
    print("\nMissing files:")
    for m in missing:
        print(f"🔻 {m}")



✅ Comparison complete.
✔️  Matched: 0 files: []
❌ Missing: 1 files: ['.DS_Store']

Missing files:
🔻 .DS_Store


In [11]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Midjourney" "s3://maddeproject/MADDE Eval Dataset/Midjourney" --delete

upload: ../../../../datasets/Midjourney/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/Black/c98a9b96-8eb4-4e7c-a674-61d2245507e3_0.png
delete: s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/SouthAsian/e41f3d51-026b-49be-af9c-61b00a3ba624_0.png
delete: s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/SouthAsian/6640450c-055f-49fb-b235-124f771ecfb6_0.png
upload: ../../../../datasets/Midjourney/Female/SouthAsian/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/SouthAsian/.DS_Store
upload: ../../../../datasets/Midjourney/Female/Black/gauravsharan_httpss.mj.runyRB97aQ03Q0_Generate_a_new_real-loo_30edee2f-b715-4ab0-aef6-99575703e185_0.png to s3://maddeproject/MADDE Eval Dataset/Midjourney/Female/Black/gauravsharan_httpss.mj.runyRB97aQ03Q0_Generate_a_new_real-loo_30edee2f-b715-4ab0-aef6-99575703e185_0.png
upload: ../../../../datasets/Midjourney/Mal

In [12]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Stylegan2" "s3://maddeproject/MADDE Eval Dataset/Stylegan2" --delete

upload: ../../../../datasets/Stylegan2/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Stylegan2/Female/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Female/Black/33703_woman_black_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Female/SouthAsian/16849_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Female/SouthAsian/17743_stylegan2.png
upload: ../../../../datasets/Stylegan2/Male/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Black/08096_man_black_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Black/09766_man_black_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Black/10369_man_black_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Black/10756_man_black_stylegan2.png
delete: s3://maddeproject/MADDE Eval Dataset/Stylegan2/Male/Blac

In [13]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Dall-E" "s3://maddeproject/MADDE Eval Dataset/Dall-E" --delete

upload: ../../../../datasets/Dall-E/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Dall-E/Female/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Female/Black/33703_woman_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Female/SouthAsian/17743_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Female/SouthAsian/16849_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Male/Black/09766_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Male/Black/08096_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Male/Black/10369_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Male/Black/10756_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Dall-E/Male/Black/10912_man_black_output.png
upload: ../../../../datasets/Dall-E/Female/SouthAsian/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Dall-E/Female/SouthAsian/.DS_Store
delete: s3://m

In [16]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/StableDiffusion" "s3://maddeproject/MADDE Eval Dataset/StableDiffusion" --delete

upload: ../../../../datasets/StableDiffusion/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Female/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Male/Black/08096_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Female/Black/33703_woman_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Female/SouthAsian/16849_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Female/SouthAsian/17743_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Male/Black/09766_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Male/Black/10369_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Male/Black/10756_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Male/Black/10912_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/StableDiffusion/Ma

In [18]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Firefly" "s3://maddeproject/MADDE Eval Dataset/Firefly" --delete

upload: ../../../../datasets/Firefly/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Firefly/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Female/Black/33703_woman_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Female/SouthAsian/16849_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Female/SouthAsian/17743_output.png
upload: ../../../../datasets/Firefly/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Firefly/Female/Black/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Male/Black/08096_man_black_output.png
upload: ../../../../datasets/Firefly/Female/SouthAsian/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Firefly/Female/SouthAsian/.DS_Store
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Male/Black/09766_man_black_output.png
delete: s3://maddeproject/MADDE Eval Dataset/Firefly/Male/Black/10369_man_black_output.png
upload: ../../../../datasets/Firefly/Male/Black/.DS_Store to s3://maddeproject/

In [19]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Real" "s3://maddeproject/MADDE Eval Dataset/Real"

upload: ../../../../datasets/Real/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real/.DS_Store
upload: ../../../../datasets/Real/Female/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real/Female/.DS_Store
upload: ../../../../datasets/Real/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real/Female/Black/.DS_Store
upload: ../../../../datasets/Real/Female/Black/03380_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real/Female/Black/03380_woman_black.png
upload: ../../../../datasets/Real/Female/Black/03978_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real/Female/Black/03978_woman_black.png
upload: ../../../../datasets/Real/Female/Black/03328_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real/Female/Black/03328_woman_black.png
upload: ../../../../datasets/Real/Female/Black/04272_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real/Female/Black/04272_woman_black.png
upload: ../../../../datasets/Real/Female/Black/03788_woman_black.pn

In [20]:
! aws s3 sync "/Users/gauravsharan/Capstone/madde/datasets/Real_for_df" "s3://maddeproject/MADDE Eval Dataset/Real_for_df"

upload: ../../../../datasets/Real_for_df/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real_for_df/.DS_Store
upload: ../../../../datasets/Real_for_df/Female/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/.DS_Store
upload: ../../../../datasets/Real_for_df/Female/Black/.DS_Store to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/Black/.DS_Store
upload: ../../../../datasets/Real_for_df/Female/Black/10439_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/Black/10439_woman_black.png
upload: ../../../../datasets/Real_for_df/Female/Black/09403_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/Black/09403_woman_black.png
upload: ../../../../datasets/Real_for_df/Female/Black/08777_woman_black.png to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/Black/08777_woman_black.png
upload: ../../../../datasets/Real_for_df/Female/Black/15438.png to s3://maddeproject/MADDE Eval Dataset/Real_for_df/Female/Black/154